# Parameters

In [1]:
import os
import sys

In [2]:
# Set global random seed
SEED = 1234

# Set the project directories for local and/or Google Colab 
dir_local = '.'
dir_colab = 'drive/MyDrive/NLP_code_notebooks/project/'
file_to_import = os.path.join(dir_colab, "byt5_model.py")

# Set the important parameters for the model
learning_rate = 1e-4
num_epochs = 50
batch_size = 16

In [3]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [4]:
if IN_COLAB:
  print('Running on Google Colab')
  from google.colab import drive

  !pip install transformers datasets
  drive.mount('/content/drive')
  dir_project = dir_colab
  !cp $file_to_import .
  #sys.path.append(dir_project)
else:
  print('Running locally')
  dir_project = dir_local

Running on Google Colab
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 35.4 MB/s eta 

# Code

## Imports, time, and random seed

In [5]:
import pandas as pd
import numpy as np
import torch
from byt5_model import *
from pathlib import Path
from datetime import datetime

In [6]:
from transformers import set_seed

In [7]:
dir_dataset = os.path.join(dir_project, 'data')

In [8]:
time_string = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [9]:
torch.manual_seed(SEED)
np.random.seed(SEED)
set_seed(SEED)

In [10]:
model_name = 'google/byt5-small'


## Data

In [11]:
header_names = ["lemma", "labels", "features"]
df_train_ger, df_valid_ger, df_test_ger = load_raw_data_as_df(dir_dataset)
df_train_tur, df_valid_tur, df_test_tur = load_raw_data_as_df(dir_dataset, which_dataset="turkish")

In [12]:
df_train_ger.head()

,lemma,labels,features,inputs
0,Plätzchen,Plätzchen,N;NOM;NEUT;PL,Plätzchen N;NOM;NEUT;PL
1,Kastanie,Kastanien,N;NOM;FEM;PL,Kastanie N;NOM;FEM;PL
2,Linie,Linien,N;NOM;FEM;PL,Linie N;NOM;FEM;PL
3,Scherz,Scherze,N;NOM;MASC;PL,Scherz N;NOM;MASC;PL
4,Wiederholung,Wiederholungen,N;NOM;FEM;PL,Wiederholung N;NOM;FEM;PL


In [13]:
df_train_tur.head()

,lemma,labels,features,inputs
0,kurtarmak,kurtarmış olacak mıydı,V;PROSP;SG;3;POS;PST;INTR;LGSPEC03,kurtarmak V;PROSP;SG;3;POS;PST;INTR;LGSPEC03
1,inmek,inecek olacak mıyım,V;PROSP;IND;SG;1;POS;FUT;INTR,inmek V;PROSP;IND;SG;1;POS;FUT;INTR
2,kaçmak,kaçmamışlarmış,V;DECL;PL;3;NEG;PST;LGSPEC01;LGSPEC03,kaçmak V;DECL;PL;3;NEG;PST;LGSPEC01;LGSPEC03
3,söz vermek,söz vermeyeceğim,V;DECL;IND;SG;1;NEG;FUT,söz vermek V;DECL;IND;SG;1;NEG;FUT
4,parlatmak,parlatmamış mısın,V;SG;2;NEG;PST;INTR;LGSPEC03,parlatmak V;SG;2;NEG;PST;INTR;LGSPEC03


In [14]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [15]:
tokenizer_ger = get_tokenizer(model_name)
tokenizer_tur = get_tokenizer(model_name)

In [44]:
tokenizer_tur.save_pretrained(os.path.join(dir_project, "saved_tokenizer_tur"))
tokenizer_ger.save_pretrained(os.path.join(dir_project, "saved_tokenizer_ger"))

('drive/MyDrive/NLP_code_notebooks/project/saved_tokenizer_ger/tokenizer_config.json',
 'drive/MyDrive/NLP_code_notebooks/project/saved_tokenizer_ger/special_tokens_map.json',
 'drive/MyDrive/NLP_code_notebooks/project/saved_tokenizer_ger/added_tokens.json')

In [16]:
tokenized_train_ger, tokenized_valid_ger, tokenized_test_ger = get_tokenized_data(tokenizer_ger, df_train_ger, df_valid_ger, df_test_ger)

In [17]:
tokenized_train_tur, tokenized_valid_tur, tokenized_test_tur = get_tokenized_data(tokenizer_tur, df_train_tur, df_valid_tur, df_test_tur)

In [18]:
tokenized_train_ger

{'input_ids': tensor([[ 83, 111, 198,  ...,   0,   0,   0],
         [ 78, 100, 118,  ...,   0,   0,   0],
         [ 79, 108, 113,  ...,   0,   0,   0],
         ...,
         [ 80, 108, 119,  ...,   0,   0,   0],
         [ 86, 119, 100,  ...,   0,   0,   0],
         [ 85, 120, 103,  ...,   0,   0,   0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[ 83, 111, 198,  ...,   0,   0,   0],
         [ 78, 100, 118,  ...,   0,   0,   0],
         [ 79, 108, 113,  ...,   0,   0,   0],
         ...,
         [ 80, 108, 119,  ...,   0,   0,   0],
         [ 86, 119, 100,  ...,   0,   0,   0],
         [ 85, 120, 103,  ...,   0,   0,   0]])}

In [19]:
tokenized_train_tur

{'input_ids': tensor([[110, 120, 117,  ...,   0,   0,   0],
         [108, 113, 112,  ...,   0,   0,   0],
         [110, 100, 198,  ...,   0,   0,   0],
         ...,
         [125, 100, 105,  ...,   0,   0,   0],
         [118, 199, 180,  ...,   0,   0,   0],
         [101, 198, 185,  ...,   0,   0,   0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[110, 120, 117,  ...,   0,   0,   0],
         [108, 113, 104,  ...,   0,   0,   0],
         [110, 100, 198,  ...,   0,   0,   0],
         ...,
         [125, 100, 105,  ...,   0,   0,   0],
         [118, 199, 180,  ...,   0,   0,   0],
         [101, 198, 185,  ...,   0,   0,   0]])}

## Fine-tuning the pretrained model for the German dataset

In [ ]:
import torch
from transformers import T5ForConditionalGeneration

In [ ]:
dir_path_model_ger = os.path.join(dir_project, "saved_model_fine_ger_new")
if not os.path.isdir(dir_path_model_ger):
  os.makedirs(dir_path_model_ger)

In [ ]:
model_fine_ger = get_byt5_model(device, model_name)

In [ ]:
optimizer_fine_ger = get_optimizer(model_fine_ger, learning_rate)

In [ ]:
train_dataloader_ger = get_dataloader(tokenized_train_ger, batch_size=batch_size)
valid_dataloader_ger = get_dataloader(tokenized_valid_ger, batch_size=1, shuffle=False)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
list_train_losses_fine_ger, list_valid_losses_fine_ger = [], []
list_train_losses_fine_ger, list_valid_losses_fine_ger = train_validation_loop(model_fine_ger, 
                                                                               train_dataloader_ger, 
                                                                               valid_dataloader_ger, 
                                                                               optimizer_fine_ger, 
                                                                               device, 
                                                                               dir_path_model_ger, 
                                                                               num_epochs=num_epochs)
np.save(os.path.join(dir_project, "train_losses_fine_ger.npy"), np.array(list_train_losses_fine_ger))
np.save(os.path.join(dir_project, "valid_losses_fine_ger.npy"), np.array(list_valid_losses_fine_ger))

epoch: 1 / 50, train loss: 35.1419, validation loss: 17.3433
epoch: 2 / 50, train loss: 12.8769, validation loss: 4.0278
epoch: 3 / 50, train loss: 4.2891, validation loss: 3.5307
epoch: 4 / 50, train loss: 3.6173, validation loss: 2.9870
epoch: 5 / 50, train loss: 3.0373, validation loss: 2.1625
epoch: 6 / 50, train loss: 2.0979, validation loss: 0.8594
epoch: 7 / 50, train loss: 0.8113, validation loss: 0.1864
epoch: 8 / 50, train loss: 0.3339, validation loss: 0.1204
epoch: 9 / 50, train loss: 0.2314, validation loss: 0.0991
epoch: 10 / 50, train loss: 0.1700, validation loss: 0.0877
epoch: 11 / 50, train loss: 0.1565, validation loss: 0.0856
epoch: 12 / 50, train loss: 0.1402, validation loss: 0.0700
epoch: 13 / 50, train loss: 0.1241, validation loss: 0.0608
epoch: 14 / 50, train loss: 0.1068, validation loss: 0.0591
epoch: 15 / 50, train loss: 0.0986, validation loss: 0.0540
epoch: 16 / 50, train loss: 0.0954, validation loss: 0.0499
epoch: 17 / 50, train loss: 0.0878, validation

## Fine-tuning the pretrained model for the Turkish dataset


In [20]:
dir_path_model_tur = os.path.join(dir_project, "saved_model_fine_tur_new")
if not os.path.isdir(dir_path_model_tur):
  os.makedirs(dir_path_model_tur)

In [22]:
model_fine_tur = get_byt5_model(device, model_name)

In [23]:
optimizer_fine_tur = get_optimizer(model_fine_tur, learning_rate)

In [24]:
train_dataloader_tur = get_dataloader(tokenized_train_tur, batch_size=batch_size)
valid_dataloader_tur = get_dataloader(tokenized_valid_tur, batch_size=1, shuffle=False)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [25]:
list_train_losses_fine_tur, list_valid_losses_fine_tur = [], []
list_train_losses_fine_tur, list_valid_losses_fine_tur = train_validation_loop(model_fine_tur, 
                                                                               train_dataloader_tur, 
                                                                               valid_dataloader_tur, 
                                                                               optimizer_fine_tur, 
                                                                               device, 
                                                                               dir_path_model_tur, 
                                                                               num_epochs=num_epochs)
np.save(os.path.join(dir_project, "train_losses_fine_tur.npy"), np.array(list_train_losses_fine_tur))
np.save(os.path.join(dir_project, "valid_losses_fine_tur.npy"), np.array(list_valid_losses_fine_tur))

epoch: 1 / 50, train loss: 30.4822, validation loss: 12.8650
epoch: 2 / 50, train loss: 7.3360, validation loss: 3.9880
epoch: 3 / 50, train loss: 3.8533, validation loss: 3.2203
epoch: 4 / 50, train loss: 3.2001, validation loss: 2.4393
epoch: 5 / 50, train loss: 2.1218, validation loss: 1.0188
epoch: 6 / 50, train loss: 0.8990, validation loss: 0.3400
epoch: 7 / 50, train loss: 0.4866, validation loss: 0.2932
epoch: 8 / 50, train loss: 0.4165, validation loss: 0.2813
epoch: 9 / 50, train loss: 0.3908, validation loss: 0.2701
epoch: 10 / 50, train loss: 0.3725, validation loss: 0.2584
epoch: 11 / 50, train loss: 0.3456, validation loss: 0.2412
epoch: 12 / 50, train loss: 0.3262, validation loss: 0.2224
epoch: 13 / 50, train loss: 0.3009, validation loss: 0.2047
epoch: 14 / 50, train loss: 0.2764, validation loss: 0.1945
epoch: 15 / 50, train loss: 0.2646, validation loss: 0.1876
epoch: 16 / 50, train loss: 0.2509, validation loss: 0.1834
epoch: 17 / 50, train loss: 0.2388, validation 